In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
os.chdir('gdrive/MyDrive/boddam-masters-project')

In [3]:
!sudo apt-get install openslide-tools
!sudo apt-get install python-openslide
!pip install openslide-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openslide-tools is already the newest version (3.4.1+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-openslide is already the newest version (1.1.1-2ubuntu4).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
"""
install
keras, numpy, tensorflow, scipy, Pillow

"""

from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import callbacks
from keras.utils import load_img, img_to_array
import pandas as pd
from PIL import Image
import glob
import numpy as np
from openslide import ImageSlide, open_slide
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from numpy import load


In [5]:
train_data_dir = "data/train"
test_data_dir = "data/test"
data_width, data_height = 1000, 1000
#nb_train_samples = 8
#nb_validation_samples = 10
nb_filters1 = 2
nb_filters2 = 4
conv1_size = 3
conv2_size = 3
pool_size = 2
classes_num = 2
batch_size = 50
lr = 0.005
epochs = 5

tile_size = 256

In [6]:
train_image_data_init = load('train_image_data.npz')
train_image_data = train_image_data_init['arr_0']
train_class_data_init = load('train_class_data.npz')
train_class_data = train_class_data_init['arr_0']

print(train_image_data.shape)
print(train_class_data.shape)


(331, 1000, 1000, 3)
(331, 2)


In [7]:
(trainX, valX, trainY, valY) = train_test_split(
    train_image_data, 
    train_class_data, 
    test_size=0.40, 
    random_state=42
)

In [8]:
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow(
    x=trainX, 
    y=trainY,
    batch_size=batch_size 
    )
valid_generator = ImageDataGenerator().flow(valX, valY,
    batch_size=batch_size )


In [12]:
test_image_data_init = load('test_image_data.npz')
test_image_data = test_image_data_init['arr_0']
test_class_data_init = load('test_class_data.npz')
test_class_data = test_class_data_init['arr_0']

print(test_image_data.shape)
print(test_class_data.shape)

(120, 1000, 1000, 3)
(120, 2)


In [10]:
model = Sequential()
model.add(Conv2D(nb_filters1, (conv1_size, conv1_size), padding="same", input_shape=(data_width, data_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
model.add(Conv2D(nb_filters1, (conv1_size, conv1_size), padding="same", input_shape=(data_width, data_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])


In [11]:
model.fit_generator(
    train_generator, 
    validation_data=valid_generator,
    epochs=5
)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/5
4/4 [==============================] - 27s 4s/step - loss: 887.0704 - accuracy: 0.6616 - val_loss: 489.8593 - val_accuracy: 0.7594
Epoch 2/5
4/4 [==============================] - 6s 2s/step - loss: 242.8013 - accuracy: 0.6061 - val_loss: 106.6304 - val_accuracy: 0.7594
Epoch 3/5
4/4 [==============================] - 6s 2s/step - loss: 102.1756 - accuracy: 0.7273 - val_loss: 1.8942 - val_accuracy: 0.9398
Epoch 4/5
4/4 [==============================] - 5s 1s/step - loss: 9.8886 - accuracy: 0.8687 - val_loss: 1.7523 - val_accuracy: 0.9474
Epoch 5/5
4/4 [==============================] - 5s 1s/step - loss: 3.9666 - accuracy: 0.9697 - val_loss: 1.5818 - val_accuracy: 0.9699


In [ ]:
tp = 0
tn = 0
fp = 0
fn = 0
#print(test_image_data.shape)
result_array = model.predict(test_image_data)
for i in range(len(test_class_data)):
  #data = [test_image_data[i]]
  #data = np.array(test_image_data[i])
  expected = test_class_data[i]
  #print(data.shape)
  result = result_array[i]
  #result = model.predict(data)
  # index 0 is normal; index 1 is tumor
  if (expected[0] == 1):
    print("Label: Normal")
    if (result[0] == expected[0]):
      print("Predicted answer: Normal")
      tp += 1
    else:
      print("Predicted answer: Tumor")
      fn += 1
  else:
    print("Label: Tumor")
    if (result[0] == expected[0]):
      print("Predicted answer: Tumor")
      tn += 1
    else:
      print("Predicted answer: Normal")
      fp += 1



In [22]:
"""
Check metrics
"""
print("True Positive: ", tp)
print("True Negative: ", tn)
print("False Positive: ", fp)
print("False Negative: ", fn)

if (tp + fp) != 0:
    precision = tp / (tp + fp)
    print("Precision: ", precision)
else:
    print("Precision cannot be calculated because (tp+fp)=0")
    print("F-measure also cannot be calculated")
if (tp + fn) != 0:
    recall = tp / (tp + fn)
    print("Recall: ", recall)
else:
    print("Recall cannot be calculated because (tp+fn)=0")
    print("F-measure also cannot be calculated")
if (tp + fp) != 0 and (tp + fn) != 0:
    f_measure = (2 * recall * precision) / (recall + precision)
    print("F-measure: ", f_measure)

True Positive:  25
True Negative:  48
False Positive:  44
False Negative:  3
Precision:  0.36231884057971014
Recall:  0.8928571428571429
F-measure:  0.5154639175257733


In [23]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow(test_image_data, test_class_data, batch_size=1)
score = model.evaluate_generator(test_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Test loss: 0.9627031683921814
Test accuracy: 0.9750000238418579
